In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/stsbenchmark')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/stsbenchmark'

**EMBEDDINGS DELLE COPPIE DI FRASI, SENTENCE TRANSFORMERS**


In [2]:
import csv
import pandas as pd
import json
import string
import re
import numpy as np
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

In [3]:
columns=['genre', 'file', 'year', 'index', 'score', 'sentence1', 'sentence2']
df_train = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-train.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')
df_dev = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-dev.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')
df_test = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-test.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')

In [4]:
def pre_processing(df: pd.DataFrame) -> pd.DataFrame:
    df['genre'] = df['genre'].replace('main-', '', regex=True)
    df['genre'] = df['genre'].replace('forum', 'forums')
    df['year'] = df['year'].replace(r'\D', '', regex=True)
    df['score'] = MinMaxScaler().fit_transform(df[['score']])
    return df

In [5]:
df_train = pre_processing(df_train)
df_dev = pre_processing(df_dev)
df_test = pre_processing(df_test)

--descrizione dataset--

**Data Processing**

In [6]:
import string
import re

regex = '[' + string.punctuation + ']'
print(regex)

[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]


In [7]:
df_train['vocab1'] = df_train['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))  
df_train['vocab2'] = df_train['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))
df_dev['vocab1'] = df_dev['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split())) 
df_dev['vocab2'] = df_dev['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))
df_test['vocab1'] = df_test['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split())) 
df_test['vocab2'] = df_test['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))

In [8]:
'''global_vocab1 = df_train['vocab1'].sum()
print(global_vocab1[:20])

global_vocab2 = df_train['vocab2'].sum()
print(global_vocab2[:20])'''

"global_vocab1 = df_train['vocab1'].sum()\nprint(global_vocab1[:20])\n\nglobal_vocab2 = df_train['vocab2'].sum()\nprint(global_vocab2[:20])"

In [9]:
'''import nltk
counts1 = nltk.Counter(global_vocab1)
print(counts1)

counts2 = nltk.Counter(global_vocab2)
print(counts2)'''

'import nltk\ncounts1 = nltk.Counter(global_vocab1)\nprint(counts1)\n\ncounts2 = nltk.Counter(global_vocab2)\nprint(counts2)'

In [10]:
'''counts1.most_common(30)

counts2.most_common(30)'''

'counts1.most_common(30)\n\ncounts2.most_common(30)'

In [11]:
'''nltk.download('stopwords')
from nltk.corpus import stopwords

# vocab 1 without stopwords
vocab1_nostop = [w for w in global_vocab1 if w not in stopwords.words('english')]
counts1_nostop = nltk.Counter(vocab1_nostop)

# vocab 2 without stopwords
vocab2_nostop = [w for w in global_vocab2 if w not in stopwords.words('english')]
counts2_nostop = nltk.Counter(vocab2_nostop)'''

"nltk.download('stopwords')\nfrom nltk.corpus import stopwords\n\n# vocab 1 without stopwords\nvocab1_nostop = [w for w in global_vocab1 if w not in stopwords.words('english')]\ncounts1_nostop = nltk.Counter(vocab1_nostop)\n\n# vocab 2 without stopwords\nvocab2_nostop = [w for w in global_vocab2 if w not in stopwords.words('english')]\ncounts2_nostop = nltk.Counter(vocab2_nostop)"

removing stopwords


In [12]:
'''from nltk.corpus import stopwords
for i in range(0,len(df_train)):
  df_train['sentence1'][i] = re.sub(regex,' ',df_train['sentence1'][i])
  df_train['sentence2'][i] = re.sub(regex,' ',df_train['sentence2'][i])
  words1 = df_train['sentence1'][i].lower().split()
  words2 = df_train['sentence2'][i].lower().split()
  df_train['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_train['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])

for i in range(0,len(df_dev)):
  df_dev['sentence1'][i] = re.sub(regex,' ',df_dev['sentence1'][i])
  df_dev['sentence2'][i] = re.sub(regex,' ',df_dev['sentence2'][i])
  words1 = df_dev['sentence1'][i].lower().split()
  words2 = df_dev['sentence2'][i].lower().split()
  df_dev['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_dev['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])

for i in range(0,len(df_test)):
  df_test['sentence1'][i] = re.sub(regex,' ',df_test['sentence1'][i])
  df_test['sentence2'][i] = re.sub(regex,' ',df_test['sentence2'][i])
  words1 = df_test['sentence1'][i].lower().split()
  words2 = df_test['sentence2'][i].lower().split()
  df_test['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_test['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])'''
 


"from nltk.corpus import stopwords\nfor i in range(0,len(df_train)):\n  df_train['sentence1'][i] = re.sub(regex,' ',df_train['sentence1'][i])\n  df_train['sentence2'][i] = re.sub(regex,' ',df_train['sentence2'][i])\n  words1 = df_train['sentence1'][i].lower().split()\n  words2 = df_train['sentence2'][i].lower().split()\n  df_train['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])\n  df_train['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])\n\nfor i in range(0,len(df_dev)):\n  df_dev['sentence1'][i] = re.sub(regex,' ',df_dev['sentence1'][i])\n  df_dev['sentence2'][i] = re.sub(regex,' ',df_dev['sentence2'][i])\n  words1 = df_dev['sentence1'][i].lower().split()\n  words2 = df_dev['sentence2'][i].lower().split()\n  df_dev['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])\n  df_dev['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])\n\nfor i in r

In [13]:
len(df_train)

5749

**Word Embeddings**

In [14]:
!pip install -U sentence-transformers
!pip install datasets
!pip install hnswlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=40f81e075dd0dffc2c2b14a0a6a66d6ca0de3fa80324323f8f4b727267a95ed3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
from matplotlib import pyplot as plt
%matplotlib inline

In [16]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-mpnet-base-v2')

In [17]:
#df_train.head()

corpus_train1= np.array(df_train['sentence1'] + ' ' + df_train['sentence2'])

corpus_dev1 = np.array(df_dev['sentence1'] + ' ' + df_dev['sentence2'])

corpus_test1= np.array(df_test['sentence1'] + ' ' + df_test['sentence2'])
len(corpus_train1)


5749

In [18]:
corpus_embeddings_train1 = model.encode(corpus_train1, convert_to_tensor=True) #each sentence into a single vector
#corpus_embeddings_train2 = model.encode(corpus_train2, convert_to_tensor=True)

corpus_embeddings_dev1 = model.encode(corpus_dev1, convert_to_tensor=True) 
#corpus_embeddings_dev2 = model.encode(corpus_dev2, convert_to_tensor=True) #each sentence into a single vector

corpus_embeddings_test1 = model.encode(corpus_test1, convert_to_tensor=True) 
#corpus_embeddings_test2 = model.encode(corpus_test2, convert_to_tensor=True) #each sentence into a single vector
corpus_embeddings_test1.size()

torch.Size([1379, 768])

In [19]:
print(corpus_embeddings_train1.size())
#print(corpus_embeddings_train2.size())


torch.Size([5749, 768])


In [20]:
X_train=corpus_embeddings_train1
X_train.size()

torch.Size([5749, 768])

In [21]:
x_dev=corpus_embeddings_dev1

X_train = torch.cat([X_train, x_dev])
X_train.size()

torch.Size([7249, 768])

In [22]:
x_test=corpus_embeddings_test1

In [23]:
'''from sklearn.manifold import TSNE

tsne_train = TSNE(n_components=3, n_iter=500, n_jobs=-1, verbose=2)
tsne_embedding_train = tsne_train.fit_transform(corpus_embeddings_train)
x, y, z = np.transpose(tsne_embedding_train)
x1, y1, z1 = np.transpose(tsne_embedding_train[:len(df_train)])
x2, y2, z2 = np.transpose(tsne_embedding_train[len(df_train):])

tsne_test = TSNE(n_components=3, n_iter=500, n_jobs=-1, verbose=2)
tsne_embedding_test = tsne_test.fit_transform(corpus_embeddings_test)
x, y, z = np.transpose(tsne_embedding_test)
x1_test, y1_test, z1_test = np.transpose(tsne_embedding_test[:len(df_test)])
x2_test, y2_test, z2_test = np.transpose(tsne_embedding_test[len(df_test):])'''

'from sklearn.manifold import TSNE\n\ntsne_train = TSNE(n_components=3, n_iter=500, n_jobs=-1, verbose=2)\ntsne_embedding_train = tsne_train.fit_transform(corpus_embeddings_train)\nx, y, z = np.transpose(tsne_embedding_train)\nx1, y1, z1 = np.transpose(tsne_embedding_train[:len(df_train)])\nx2, y2, z2 = np.transpose(tsne_embedding_train[len(df_train):])\n\ntsne_test = TSNE(n_components=3, n_iter=500, n_jobs=-1, verbose=2)\ntsne_embedding_test = tsne_test.fit_transform(corpus_embeddings_test)\nx, y, z = np.transpose(tsne_embedding_test)\nx1_test, y1_test, z1_test = np.transpose(tsne_embedding_test[:len(df_test)])\nx2_test, y2_test, z2_test = np.transpose(tsne_embedding_test[len(df_test):])'

In [24]:
'''#Mean Average of distances
X_train= pd.DataFrame(columns=['datax','datay','dataz'])
for i in range(0,len(df_train)):
  dist_x=np.sqrt(abs(x1[i]*x1[i] - x2[i]*x2[i]))
  dist_y=np.sqrt(abs(y1[i]*y1[i] - y2[i]*y2[i]))
  dist_z=np.sqrt(abs(z1[i]*z1[i] - z2[i]*z2[i]))
  #coord_avg=np.mean([dist_x,dist_y,dist_z])
  new_row={'datax': dist_x, 'datay': dist_y, 'dataz': dist_z}
  X_train.loc[len(X_train)]=new_row

X_test=pd.DataFrame(columns=['datax','datay','dataz'])
for i in range(0,len(df_test)):
  dist_x=np.sqrt(abs(x1_test[i]*x1_test[i] - x2_test[i]*x2_test[i]))
  dist_y=np.sqrt(abs(y1_test[i]*y1_test[i] - y2_test[i]*y2_test[i]))
  dist_z=np.sqrt(abs(z1_test[i]*z1_test[i] - z2_test[i]*z2_test[i]))
  #coord_avg=np.mean([dist_x,dist_y,dist_z])
  new_row={'datax': dist_x, 'datay': dist_y, 'dataz': dist_z}
  X_test.loc[len(X_test)]=new_row

print(X_train.head(15),X_test.head(15))'''

"#Mean Average of distances\nX_train= pd.DataFrame(columns=['datax','datay','dataz'])\nfor i in range(0,len(df_train)):\n  dist_x=np.sqrt(abs(x1[i]*x1[i] - x2[i]*x2[i]))\n  dist_y=np.sqrt(abs(y1[i]*y1[i] - y2[i]*y2[i]))\n  dist_z=np.sqrt(abs(z1[i]*z1[i] - z2[i]*z2[i]))\n  #coord_avg=np.mean([dist_x,dist_y,dist_z])\n  new_row={'datax': dist_x, 'datay': dist_y, 'dataz': dist_z}\n  X_train.loc[len(X_train)]=new_row\n\nX_test=pd.DataFrame(columns=['datax','datay','dataz'])\nfor i in range(0,len(df_test)):\n  dist_x=np.sqrt(abs(x1_test[i]*x1_test[i] - x2_test[i]*x2_test[i]))\n  dist_y=np.sqrt(abs(y1_test[i]*y1_test[i] - y2_test[i]*y2_test[i]))\n  dist_z=np.sqrt(abs(z1_test[i]*z1_test[i] - z2_test[i]*z2_test[i]))\n  #coord_avg=np.mean([dist_x,dist_y,dist_z])\n  new_row={'datax': dist_x, 'datay': dist_y, 'dataz': dist_z}\n  X_test.loc[len(X_test)]=new_row\n\nprint(X_train.head(15),X_test.head(15))"

In [25]:
#y_train = df_train['score']
y_train = pd.concat([df_train['score'],df_dev['score']]).reset_index(drop=True)

In [26]:
y_dev = df_dev['score']

In [27]:
y_test = df_test['score']

len(y_train)

7249

**Training Data**

In [28]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [29]:
y_pred = model.predict(x_test)

In [30]:
#print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(x_test))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

#print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MSE train:  0.033194080536707575
RMSE train:  0.18219242722107737
r2:  0.6201430122203674
MSE test:  0.05207648603516488
RMSE test:  0.22820273012206688
r2:  0.44016193775633594


In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
# Add traces
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Results"
)

# Set x-axis title
fig.update_xaxes(title_text="Couple of sentences")

# Set y-axes titles
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [32]:
from sklearn.linear_model import PoissonRegressor

model = PoissonRegressor()
model.fit(X_train, y_train)

PoissonRegressor()

In [33]:
y_pred = model.predict(x_test)

In [34]:
print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MAE train:  0.25279438166235735
MSE train:  0.08713479726422668
RMSE train:  0.29518603839651136
r2:  0.0028715637122120885
MAE test:  0.26103884200811284
MSE test:  0.09278434019193561
RMSE test:  0.30460522023093367
r2:  0.0025401256042009157


In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [36]:
from sklearn.svm import SVR
'''model = SVR()
parameters = {'C': [10],
             'epsilon': [0.1, 0.01],
             'gamma':['auto'],
             'kernel': ['linear', 'poly','rbf'],
             'degree': [2,3,5]
             }

#DEFINE YOUR GRIDSEARCH 
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, parameters, cv=3, verbose = 0)
gs_dev = GridSearchCV(model, parameters, cv=3, verbose = 0)

gs = gs.fit(X_train,y_train)
gs_dev = gs_dev.fit(x_dev,y_dev)'''

"model = SVR()\nparameters = {'C': [10],\n             'epsilon': [0.1, 0.01],\n             'gamma':['auto'],\n             'kernel': ['linear', 'poly','rbf'],\n             'degree': [2,3,5]\n             }\n\n#DEFINE YOUR GRIDSEARCH \nfrom sklearn.model_selection import GridSearchCV\ngs = GridSearchCV(model, parameters, cv=3, verbose = 0)\ngs_dev = GridSearchCV(model, parameters, cv=3, verbose = 0)\n\ngs = gs.fit(X_train,y_train)\ngs_dev = gs_dev.fit(x_dev,y_dev)"

In [37]:
'''print('**GRIDSEARCH RESULTS**')
print("Best score: %f using %s" % (gs.best_score_, gs.best_params_))
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
params = gs.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print("Best score: %f using %s" % (gs_dev.best_score_, gs_dev.best_params_))
means = gs_dev.cv_results_['mean_test_score']
stds = gs_dev.cv_results_['std_test_score']
params = gs_dev.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#Returns the coefficient of determination R^2 of the prediction.
#Explained variance score: 1 is perfect prediction'''


'print(\'**GRIDSEARCH RESULTS**\')\nprint("Best score: %f using %s" % (gs.best_score_, gs.best_params_))\nmeans = gs.cv_results_[\'mean_test_score\']\nstds = gs.cv_results_[\'std_test_score\']\nparams = gs.cv_results_[\'params\']\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n\nprint("Best score: %f using %s" % (gs_dev.best_score_, gs_dev.best_params_))\nmeans = gs_dev.cv_results_[\'mean_test_score\']\nstds = gs_dev.cv_results_[\'std_test_score\']\nparams = gs_dev.cv_results_[\'params\']\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n\n#Returns the coefficient of determination R^2 of the prediction.\n#Explained variance score: 1 is perfect prediction'

In [38]:
model = SVR(C= 10, degree= 2, epsilon= 0.1, gamma= 'auto', kernel= 'linear')
model.fit(X_train, y_train)

SVR(C=10, degree=2, gamma='auto', kernel='linear')

In [39]:
y_pred = model.predict(x_test)

In [40]:
#print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

#print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MAE train:  0.14687982031887437
MSE train:  0.03443601289021392
RMSE train:  0.1855694287597338
r2:  0.6059309396097916
MAE test:  0.17615308309534167
MSE test:  0.05395811755618081
RMSE test:  0.23228886662124126
r2:  0.41993382666852497


In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [42]:
from sklearn import linear_model
from sklearn.linear_model import BayesianRidge
model = linear_model.BayesianRidge()
model.fit(X_train, y_train)

BayesianRidge()

In [43]:
y_pred = model.predict(x_test)

In [44]:
#print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

#print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MAE train:  0.150882042569276
MSE train:  0.034918837694146485
RMSE train:  0.18686582805357024
r2:  0.6004057262982103
MAE test:  0.17346676409827264
MSE test:  0.050887563026882514
RMSE test:  0.22558271881259548
r2:  0.45294322166754797


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()